## Random Forest
Random Forest is an ensemble learning algorithm that builds multiple decision trees and combines their outputs to improve accuracy and reduce overfitting. It is used for both classification and regression tasks.

It works by:

1. Creating multiple decision trees using different random subsets of the training data (Bootstrap Aggregation or Bagging).
2. Selecting a random subset of features at each split to ensure diversity among trees.
3. Aggregating the predictions from all trees (majority voting for classification, averaging for regression).

### Assumptions
1. The dataset has enough diversity so that multiple trees can learn different patterns.
2. Relationships between features and target variables can be captured through multiple decision boundaries.
3. More trees generally improve performance but come at a computational cost.

### Limitations
1. Computationally expensive – Training many trees requires more memory and time.
2. Less interpretable than a single decision tree.
3. Prone to overfitting with noisy data if the number of trees is too large.
4. Not ideal for extremely high-dimensional data (may require feature selection).
### When to use?
1. High accuracy – When improving predictive performance is the priority.
2. Handling missing data – Can work well even with missing values.
3. Large datasets – Works effectively with large datasets with many features.
4. Reducing overfitting – More robust than a single decision tree, making it suitable for complex problems.